# Import e dados

In [1]:
!python -m spacy download pt_core_news_sm

2023-12-26 21:34:04.562115: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 21:34:04.562477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 21:34:04.564945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-26 21:34:04.628354: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-26 21:34:09.949313: W tensorflow/compiler/tf2

In [2]:
import pandas as pd
import spacy
from gensim.models import Word2Vec
import logging



path_treino = "/content/drive/MyDrive/w2v-Alura/treino_caelum.csv"
path_teste = "/content/drive/MyDrive/w2v-Alura/teste_caelum.csv"

In [3]:
dados_treino = pd.read_csv(path_treino)
dados_treino.sample(5)

,title,text,date,category,subcategory,link
43181,Ônibus do Fenerbahçe é atacado a tiros após go...,A goleada do Fenerbahçe sobre o Rizespor por 5...,2015-04-04,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/04/1...
31362,Procuradoria pede prisão de quatro policiais f...,O MPF (Ministério Público Federal) no Rio pedi...,2016-07-04,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/04...
62366,Panelas para todos,Passar o fim de semana com filhos ainda pequen...,2015-11-03,colunas,marcelocoelho,http://www1.folha.uol.com.br/colunas/marceloco...
27722,Primeiro trailer da cinebiografia do cantor in...,O primeiro trailer da cinebiografia do cantor ...,2017-01-07,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/07...
51677,"\n\t\tRio 2016 Committee to Distribute 115,000...",FROM RIO Given that athletes are backing out ...,2016-05-07,esporte,olimpiada-no-rio,http://www1.folha.uol.com.br/esporte/olimpiada...


In [4]:
dados_teste = pd.read_csv(path_teste)
dados_teste.sample(5)

,title,text,date,category,subcategory,link
16753,As pessoas mais felizes e bonitas são as fora ...,Por que você não faz uma cirurgia para corrigi...,2016-12-27,colunas,miriangoldenberg,http://www1.folha.uol.com.br/colunas/miriangol...
14645,Mercado acionário da China congela em meio a c...,O mercado acionário chinês mostrou sinais de c...,2015-08-07,mercado,NaN,http://www1.folha.uol.com.br/mercado/2015/07/1...
17688,Governo estuda aumentar valor para novo leilão...,"O ministro de Minas e Energia, Eduardo Braga (...",2015-08-28,mercado,NaN,http://www1.folha.uol.com.br/mercado/2015/08/1...
654,Casal cria editora especializada em livros LGB...,Nos quase dez anos de carreira no mercado edit...,2015-10-20,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2015/10...
17173,Acordo para congelar produção de petróleo frac...,Um acordo para congelar os níveis de produção ...,2016-04-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/04/1...


In [5]:
nlp = spacy.load("pt_core_news_sm")

In [6]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)
doc

Rio de Janeiro é uma cidade maravilhosa

In [7]:
type(doc[2])

spacy.tokens.token.Token

# Pré-processamento

Neste caso, podemos comparar os generator expression ao list comprehension, com a diferença que os generator “criam” seus elemento apenas quando são invocados. Como o generator produz um item por vez, ele pode levar a uma grande economia de memória.

In [8]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [9]:
print(type(doc[0]), type(doc[0].text))

<class 'spacy.tokens.token.Token'> <class 'str'>


In [10]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            # token.texto retorna um tipo string, em vez de token
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

In [11]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

## Otimização do tratamento

In [12]:
from time import time
t0 = time()

textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                         # usa todos os núcleos do processador
                                                        n_process = -1)]

tf = time() - t0
print(tf/60)


6.324458626906077


In [13]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


## Hiperparâmetros

In [14]:
w2v_modelo = Word2Vec(
                      # Skip Gram: 0 ou 1? Sim ou não?
                      sg = 0,
                      # Contexto: Quantas palavras antes e depois?
                      window = 2,
                      # Tamanho fixo
                      vector_size = 300,
                      # Frequência mínima (menos que isso é desconsiderado)
                      min_count = 5,
                      # taxa de aprendizagem
                      alpha = 0.03,
                      # alpha decai a medida que chega perto, seu valor min é...
                      min_alpha = 0.007
                      )

In [15]:
w2v_modelo

In [16]:
print(len(titulos_tratados))

# Remove linhas vazias e duplicadas
titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [17]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [18]:
# Acompanhar o que tá acontecendo
logging.basicConfig(
                    # Em que momento, a mensagem
                    format="%(asctime)s : - %(message)s",
                    # Informativo
                    level = logging.INFO
                    )

In [19]:
# CTRL C ... V
w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

# Constrói vocab
# "progress_per": de quantas em quantas iter vai me informar
w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

# Treinamento

In [25]:
# Todos atributos e métodos do modelo
print(dir(w2v_modelo))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_check_corpus_sanity', '_check_training_sanity', '_clear_post_train', '_do_train_epoch', '_do_train_job', '_get_next_alpha', '_get_thread_working_mem', '_job_producer', '_load_specials', '_log_epoch_end', '_log_epoch_progress', '_log_progress', '_log_train_end', '_raw_word_count', '_save_specials', '_scan_vocab', '_smart_save', '_train_epoch', '_train_epoch_corpusfile', '_worker_loop', '_worker_loop_corpusfile', 'add_lifecycle_event', 'add_null_word', 'alpha', 'batch_words', 'build_vocab', 'build_vocab_from_freq', 'cbow_mean', 'comment', 'compute_loss', 'corpus_count', 'corpus_total_words', 'create_binary_tree', 'cum_table', '

In [20]:
# Qtd de amostras p construir o vocab
w2v_modelo.corpus_count

84466

In [21]:
w2v_modelo.train(
                  # sentença
                  lista_lista_tokens,
                  total_examples=w2v_modelo.corpus_count,
                  # A cada 30 calcula o loss e atualiza alpha (lr)
                  epochs = 30
                  )

(14584317, 16207260)

In [22]:
w2v_modelo.wv.most_similar("google")

[('apple', 0.5856451392173767),
 ('facebook', 0.5185964703559875),
 ('waze', 0.5067850351333618),
 ('amazon', 0.49891752004623413),
 ('uber', 0.4708511233329773),
 ('volkswagen', 0.4660259187221527),
 ('yahoo', 0.46379515528678894),
 ('airbnb', 0.4503366947174072),
 ('walmart', 0.4457901418209076),
 ('tesla', 0.4427758753299713)]

In [23]:
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5579906105995178),
 ('amazon', 0.5464052557945251),
 ('buffett', 0.5268839001655579),
 ('linkedin', 0.5227991938591003),
 ('sky', 0.5226564407348633),
 ('sony', 0.5212424993515015),
 ('premium', 0.49545595049858093),
 ('syngenta', 0.49212831258773804),
 ('inbev', 0.4888550937175751),
 ('braskem', 0.48869410157203674)]

In [26]:
w2v_modelo.wv.most_similar("guitarra")


[('instrumental', 0.7244653105735779),
 ('lambert', 0.7115808725357056),
 ('cinebiografia', 0.7113637924194336),
 ('brega', 0.7113300561904907),
 ('hendrix', 0.7067840695381165),
 ('almir', 0.7067392468452454),
 ('roqueiro', 0.7045936584472656),
 ('blues', 0.7040793895721436),
 ('gentil', 0.7016094326972961),
 ('genialidade', 0.6959607601165771)]

In [28]:
w2v_modelo.wv.most_similar("filme")

[('documentário', 0.49486953020095825),
 ('drama', 0.4016474783420563),
 ('schumer', 0.382025808095932),
 ('guardiões', 0.373176246881485),
 ('divertida', 0.37038007378578186),
 ('atuações', 0.37000563740730286),
 ('seriado', 0.3655953109264374),
 ('longa', 0.35917386412620544),
 ('curta', 0.35617658495903015),
 ('fassbender', 0.3549515902996063)]

In [30]:
w2v_modelo.wv.most_similar("tesla")

[('renault', 0.6141514182090759),
 ('sony', 0.5919521450996399),
 ('amazon', 0.5849220752716064),
 ('braskem', 0.5819078087806702),
 ('gm', 0.5724146962165833),
 ('samsung', 0.566633403301239),
 ('embraer', 0.5537448525428772),
 ('toyota', 0.5477871894836426),
 ('unilever', 0.5314266681671143),
 ('apple', 0.5310393571853638)]

Track do loss

In [35]:
from gensim.models.callbacks import CallbackAny2Vec
# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
      print('Loss após a época {}: {}'.format(self.epoch, loss))
    else:
      print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss


In [32]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [33]:
w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

In [36]:
w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30,
                  compute_loss = True,
                  callbacks=[callback()])

Loss após a época 0: 1985080.25
Loss após a época 1: 1435035.25
Loss após a época 2: 1239235.0
Loss após a época 3: 1144052.5
Loss após a época 4: 1043047.0
Loss após a época 5: 1066092.5
Loss após a época 6: 964073.5
Loss após a época 7: 878606.0
Loss após a época 8: 898953.0
Loss após a época 9: 741546.0
Loss após a época 10: 764817.0
Loss após a época 11: 744162.0
Loss após a época 12: 805544.0
Loss após a época 13: 743791.0
Loss após a época 14: 687784.0
Loss após a época 15: 672528.0
Loss após a época 16: 694950.0
Loss após a época 17: 553437.0
Loss após a época 18: 506558.0
Loss após a época 19: 491590.0
Loss após a época 20: 482562.0
Loss após a época 21: 473886.0
Loss após a época 22: 441518.0
Loss após a época 23: 431002.0
Loss após a época 24: 447012.0
Loss após a época 25: 414812.0
Loss após a época 26: 408210.0
Loss após a época 27: 379354.0
Loss após a época 28: 397360.0
Loss após a época 29: 390686.0


(14584878, 16207260)

In [40]:
w2v_modelo_sg.wv.most_similar("google")

[('reguladores', 0.41892632842063904),
 ('android', 0.4046204686164856),
 ('waze', 0.3845231533050537),
 ('anunciantes', 0.38036206364631653),
 ('toshiba', 0.36562004685401917),
 ('facebook', 0.36367568373680115),
 ('apple', 0.3632224500179291),
 ('automóveis', 0.3615291118621826),
 ('buffett', 0.35875338315963745),
 ('patentes', 0.3576531410217285)]

In [41]:
w2v_modelo_sg.wv.most_similar("tesla")

[('chips', 0.5564734935760498),
 ('toyota', 0.5379801392555237),
 ('audi', 0.5206497311592102),
 ('autoguiados', 0.5199097394943237),
 ('reestrutura', 0.5054185390472412),
 ('airbags', 0.49796512722969055),
 ('dupont', 0.49553215503692627),
 ('dow', 0.49241605401039124),
 ('fertilizantes', 0.48336103558540344),
 ('fabricar', 0.47956904768943787)]

In [42]:
w2v_modelo_sg.wv.most_similar("filme")

[('telecine', 0.4182347059249878),
 ('sofia', 0.4065856337547302),
 ('locarno', 0.4043683111667633),
 ('dramalhão', 0.4024307429790497),
 ('superman', 0.3926352560520172),
 ('dirigido', 0.3918188512325287),
 ('spielberg', 0.3911934494972229),
 ('spirit', 0.39097508788108826),
 ('dirigirá', 0.38838136196136475),
 ('amos', 0.38662442564964294)]

In [43]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/w2v-Alura/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/w2v-Alura/modelo_skipgram.txt", binary=False)
